## Python 코드 정리

- 테이블 편집/제작/분석

In [ ]:
# ------------------------------------------------------------
#  필요한 라이브러리 임포트
# ------------------------------------------------------------
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# ------------------------------------------------------------
#  MySQL 연결 정보 입력
# ------------------------------------------------------------
user = 'xxxx'                  # MySQL 사용자명
password = 'xxxxxx!'      # MySQL 비밀번호
host = 'xxxxx'             # MySQL 서버 주소 (또는 127.0.0.1)
port = xxxx                    # 기본 포트
database = 'xxxx'             # 데이터베이스 이름

# 비밀번호 URL 인코딩 (특수문자 포함 대비)
password = quote_plus(password)

# SQLAlchemy 엔진 생성
engine = create_engine(
    f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4"
)

# ------------------------------------------------------------
#  쿼리 실행 (데이터 가져오기)
# ------------------------------------------------------------
query = """
SELECT *
FROM final.device_user_id_summary;
"""

# pandas로 데이터 전체 불러오기
df = pd.read_sql(query, con=engine)

# ------------------------------------------------------------
#  기본 정보 출력
# ------------------------------------------------------------
print("MySQL 연결 및 데이터 로드 성공!")
print(f"총 행 수: {len(df):,}")
print("컬럼 목록:", list(df.columns))
print("\n데이터 미리보기:")
display(df.head())

In [ ]:
# ----------------------------
# session_count 분석
# ----------------------------

# 전체 행 수
total_rows = len(df)

# session_count가 2 이상인 행만 필터링
multi_session_df = df[df['session_count'] >= 2]

# 조건 만족 행 수
multi_session_count = len(multi_session_df)

# 전체 대비 비율 계산
ratio_pct = round((multi_session_count / total_rows) * 100, 2)

# 결과 출력
print(f"총 행 수: {total_rows:,}")
print(f"session_count ≥ 2 인 행 수: {multi_session_count:,}")
print(f"비율: {ratio_pct}%")

In [ ]:
# ------------------------------------------------------------
#  CSV로 저장 (진행바 표시 + 경로 고정)
# ------------------------------------------------------------
from tqdm import tqdm
import numpy as np
import os

# tqdm에서 progress_apply 기능 활성화
tqdm.pandas()

# ------------------------------------------------------------
#  저장 경로 설정 (Desktop 고정)
# ------------------------------------------------------------
output_dir = r"C:\xxxx\xxxx\xxxx"  # 저장 폴더
output_filename = "device_user_id_summary.csv"
output_path = os.path.join(output_dir, output_filename)

# ------------------------------------------------------------
#  Chunk 단위로 저장 (진행률 표시)
# ------------------------------------------------------------
chunk_size = 10000  # 한 번에 저장할 행 수
total_rows = len(df)
num_chunks = int(np.ceil(total_rows / chunk_size))

print(f"총 {total_rows:,}행을 {num_chunks}개의 청크로 나눠 CSV로 저장합니다.")
print(f"저장 경로: {output_path}\n")

# 첫 번째 chunk는 header 포함
for i in tqdm(range(num_chunks), desc="CSV 저장 진행중"):
    start = i * chunk_size
    end = min(start + chunk_size, total_rows)
    mode = 'w' if i == 0 else 'a'
    header = True if i == 0 else False
    df.iloc[start:end].to_csv(output_path, index=False, mode=mode, header=header, encoding='utf-8-sig')

print("\nCSV 저장 완료!")
print(f"파일 위치: {output_path}")


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from tqdm import tqdm
import numpy as np

print(" MySQL 연결 완료\n")

# ------------------------------------------------------------
#  session_count = 1 인 세션 목록 불러오기
# ------------------------------------------------------------
print("1단계: session_count = 1 세션 목록 불러오는 중...")
session_list = pd.read_sql("""
    SELECT session_id
    FROM final.device_user_id_summary
    WHERE session_count = 1
""", engine)
print(f"   → 총 {len(session_list):,}개의 session_id 불러옴\n")

# ------------------------------------------------------------
#  hackle_events에서 해당 세션만 가져오기 (chunk로 나눠서 진행바 표시)
# ------------------------------------------------------------
print("2단계: hackle_events 테이블에서 해당 세션 데이터 불러오는 중...")

# 세션 ID를 문자열 리스트로 변환
session_ids = list(session_list['session_id'])
chunk_size = 5000  # 한 번에 가져올 세션 개수 (필요시 조정)

chunks = [session_ids[i:i+chunk_size] for i in range(0, len(session_ids), chunk_size)]
events_list = []

for chunk in tqdm(chunks, desc="데이터 읽는 중", unit="chunk"):
    ids = "', '".join(chunk)
    query = f"""
        SELECT session_id, event_key, event_datetime
        FROM final.hackle_events
        WHERE session_id IN ('{ids}')
        ORDER BY session_id, event_datetime;
    """
    part_df = pd.read_sql(query, engine)
    events_list.append(part_df)

events = pd.concat(events_list, ignore_index=True)
print(f"   → 총 {len(events):,}개의 이벤트 로드 완료\n")

# ------------------------------------------------------------
#  각 세션별 event_flow 생성 (시간순)
# ------------------------------------------------------------
print("3단계: 각 세션별 event_flow 생성 중...")
flow_df = (
    events.groupby('session_id')['event_key']
    .progress_apply(lambda x: ','.join(x))  # tqdm 확장으로 진행바 표시
    .reset_index()
    .rename(columns={'event_key': 'event_flow'})
)

print(f"   → {len(flow_df):,}개의 세션에 대한 event_flow 생성 완료\n")

# ------------------------------------------------------------
#  결과를 다시 MySQL로 저장
# ------------------------------------------------------------
print("4단계: 결과를 MySQL 테이블(final.session_count_1_log)로 저장 중...")
flow_df.to_sql('session_count_1_log', con=engine, schema='final', if_exists='replace', index=False)
print("저장 완료! → final.session_count_1_log 테이블 생성됨\n")


In [ ]:
from tqdm import tqdm
tqdm.pandas()  # 필수 등록 (이거 때문에 에러났던 것)

print("3단계: 각 세션별 event_flow 생성 중...")
flow_df = (
    events.groupby('session_id')['event_key']
    .progress_apply(lambda x: ','.join(x))
    .reset_index()
    .rename(columns={'event_key': 'event_flow'})
)

print(f"   → {len(flow_df):,}개의 세션에 대한 event_flow 생성 완료\n")

print("4단계: 결과를 MySQL 테이블(final.session_count_1_log)로 저장 중...")
flow_df.to_sql('session_count_1_log', con=engine, schema='final', if_exists='replace', index=False)
print("저장 완료! → final.session_count_1_log 테이블 생성됨\n")

In [ ]:
# ------------------------------------------------------------
#  안정형 MySQL 저장 (ROLLBACK 해결 + LONGTEXT + CHUNKSIZE)
# ------------------------------------------------------------

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import LONGTEXT   # ✅ 수정 포인트 (여기!)
# ↑ 'sqlalchemy.types' 가 아니라 'sqlalchemy.dialects.mysql' 에서 가져와야 함

# MySQL 연결 완전 재설정 (rollback 상태 초기화)
try:
    engine.dispose()
except:
    pass

user = 'xxxx'
password = 'xxxx'
host = 'xxxx'
port = xxxx
database = 'xxxx'

engine = create_engine(
    f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}",
    pool_pre_ping=True,             # 연결 체크
    pool_reset_on_return=None       # rollback 상태 무시
)

print("MySQL 연결 완전 재설정 완료 (rollback cleared)\n")

# flow_df를 안전하게 저장 (LONGTEXT + 분할 업로드)
flow_df.to_sql(
    'session_count_1_log',
    con=engine,
    schema='final',
    if_exists='replace',    # 기존 테이블 덮어쓰기
    index=False,
    chunksize=5000,         # 5천행 단위로 나눠 업로드
    method='multi',         # 다중 INSERT로 성능 개선
    dtype={'event_flow': LONGTEXT}  #  event_flow를 LONGTEXT로 지정
)

print("저장 완료! → final.session_count_1_log 테이블 생성됨 (event_flow: LONGTEXT)\n")

# 테이블 생성 결과 확인
check_query = "SHOW CREATE TABLE final.session_count_1_log;"
check_df = pd.read_sql(check_query, engine)
print(check_df.iloc[0, 1])


In [ ]:
# event_flow가 NULL이 아닌 행만 불러오기
df = pd.read_sql("""
    SELECT num, event_flow
    FROM final.session_count_1_log_e
    WHERE event_flow IS NOT NULL
""", engine)

# 각 행별로 연속 중복 제거 후 count 계산
def count_unique_events(flow):
    events = flow.split(',')
    compressed = [events[0]]
    for e in events[1:]:
        if e != compressed[-1]:
            compressed.append(e)
    return len(compressed)

df["event_count"] = df["event_flow"].apply(count_unique_events)

# 다시 MySQL에 저장
df[["num", "event_count"]].to_sql(
    "session_count_1_log_e_update",
    con=engine, schema="final", if_exists="replace", index=False
)


- 패턴 분석 시 사용된 파이썬 코드

import pandas as pd
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

In [ ]:
# 1. MySQL 연결 정보 입력

user = "xxxx"
password = 'xxxx'
host = "xxxxxx"    # 연결 안되면 외부IP로 변경
port = xxxx
database = "xxxx"

engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}",
    pool_pre_ping=True,
    pool_recycle=3600,
)


# 2. session_01 테이블 로드

query = """
SELECT
    num, device_id, user_id, osname, session_id,
    session_count
FROM final.session_01
"""

df = pd.read_sql(query, engine)


# 3. 결과 확인

print(f"로드된 행 수: {len(df):,}")

In [ ]:
# 3. hackle_events 테이블 로드

query = """
SELECT
    event_id,
    session_id,
    event_datetime,
    event_key
FROM final.hackle_events;
"""

df2 = pd.read_sql(query, engine)


# 3. 결과 확인
print(f"로드된 행 수: {len(df2):,}")

In [ ]:
# 제외할 이벤트 키 목록 (카테고리 기반 정리)
exclude_keys = {

    # --------------------------------
    # 세션 / 시스템 이벤트
    # --------------------------------
    '$session_end',                # 세션 종료
    '$session_start',              # 세션 시작
    'button',                      # 의미 불명확한 일반 버튼 이벤트

    # --------------------------------
    # 회원가입 / 로그인 관련
    # --------------------------------
    'complete_signup',             # 회원가입 완료
    'view_signup',                 # 회원가입 화면 표시
    'view_login',                  # 로그인 화면 표시
    'click_autoadd_contact',       # 친구 자동 추가 기능 관련

    # --------------------------------
    # 하단 탭 이동 이벤트
    # --------------------------------
    'view_lab_tap',                # 실험실 탭 진입
    'view_timeline_tap',          # 타임라인 탭 진입
    'click_bottom_navigation_questions',   # 하단 네비 - 질문
    'click_bottom_navigation_profile',     # 하단 네비 - 프로필
    'click_bottom_navigation_lab',         # 하단 네비 - 실험실
    'click_bottom_navigation_timeline',    # 하단 네비 - 타임라인

    # --------------------------------
    # 상단바 / 프로필 이동 이벤트
    # --------------------------------
    'view_profile_tap',            # 프로필 화면 진입
    'click_appbar_setting',        # 설정 진입
    'click_appbar_chat_rooms',     # 채팅방 이동

    # --------------------------------
    # 알림 / 공유 관련 이벤트
    # --------------------------------
    'click_notice',                # 공지 클릭
    'click_notice_detail',         # 공지 상세 보기
    'click_copy_profile_link_profile',
    'click_copy_profile_link_ask',
    'click_question_share',             # 질문 공유하기

    # --------------------------------
    # 랜덤 질문 기능 관련
    # --------------------------------
    'click_random_ask_shuffle',    # 랜덤 질문 다시 뽑기
    'click_random_ask_other',      # 다른 사람 랜덤 질문
    'click_random_ask_normal',     # 일반 랜덤 질문 진입
    'skip_question',               # 질문 건너뛰기

    # --------------------------------
    # 친구 추가 / 초대
    # --------------------------------
    'click_friend_invite',         # 친구 초대
    'click_invite_friend',         # 친구 초대 기능
    'view_friendplus_tap',         # 친구 확장 탭
    'click_appbar_friend_plus',

    # --------------------------------
    # 간단한 참여형 기능
    # --------------------------------
    'click_attendance'            # 출석체크
}


# session_id set 생성 (session_01 기준)
session_set = set(df['session_id'])

# df3 생성: 이벤트 필터링
df3 = df2[
    (df2['session_id'].isin(session_set)) &
    (~df2['event_key'].isin(exclude_keys))
][[
    'event_id', 'session_id', 'event_datetime', 'event_key'
]].rename(columns={
    'session_id': 'session_id_01',
    'event_datetime': 'event_datetime_01',
    'event_key': 'event_key_01'
})

print(" df3 1차 생성 완료! (user_id 아직 미포함)")
print(f"총 행 수: {len(df3):,}")

#  user_id 조인을 위한 df 준비
df_user = df[['session_id', 'user_id']].rename(
    columns={'session_id': 'session_id_01'}
)

#  user_id 조인 적용
df3 = df3.merge(df_user, on='session_id_01', how='left')

print("df3 최종 생성 완료! (user_id 포함)")

In [ ]:
# 1. 시간 기준 정렬
df3_sorted = df3.sort_values(
    ['session_id_01', 'event_datetime_01', 'event_id'],
    kind='mergesort'
)

# 2. 정렬 후 인덱스 재설정
df3_sorted = df3_sorted.reset_index(drop=True)

print("df3_sorted 생성 완료! (시간 정렬 + 인덱스 재정렬 완료)")
print(f"총 행 수: {len(df3_sorted):,}")

In [ ]:
import numpy as np

# STEP 1: 시간 차이 계산 (분 단위)
df3_sorted['time_diff_min'] = (
    df3_sorted.groupby(['user_id', 'session_id_01'])['event_datetime_01']
    .diff()
    .dt.total_seconds() / 60  # 분 단위로 변환
)

print("time_diff_min 계산 완료!")
df3_sorted[['user_id', 'session_id_01', 'event_datetime_01', 'time_diff_min']]

In [ ]:
# 90분 기준 세션 분리 flag
df3_sorted['new_session_flag'] = df3_sorted['time_diff_min'] >= 90
df3_sorted['new_session_flag'] = df3_sorted['new_session_flag'].fillna(False)

print("new_session_flag 생성 완료!")
df3_sorted[['user_id', 'session_id_01', 'event_datetime_01', 'time_diff_min', 'new_session_flag']]

In [ ]:
df3_sorted['session_split_no'] = (
    df3_sorted.groupby(['user_id', 'session_id_01'])['new_session_flag']
    .cumsum()
)

In [ ]:
df3_sorted['real_session_id'] = (
    df3_sorted['user_id'].astype(str) + '-' +
    df3_sorted['session_split_no'].astype(str)
)

print("real_session_id 생성 완료!")
df3_sorted

In [ ]:
# real_session_id 기준 event_flow 생성
grouped_flow = (
    df3_sorted
    .groupby('real_session_id')['event_key_01']
    .agg(event_flow=lambda x: ','.join(x),
         event_count='count')
    .reset_index()
)

print("grouped_flow 생성 완료!")
print(f"총 real_session_id 수: {len(grouped_flow):,}")

In [ ]:
# 기존 df에서 merge용 key 준비
df_user = df[['session_id', 'user_id', 'device_id', 'osname']].rename(
    columns={'session_id': 'session_id_01'}
)

# df3_sorted에서 real_session_id ~ session_id_01 연결 관계 가져오기
session_map = df3_sorted[['real_session_id', 'session_id_01', 'user_id']].drop_duplicates()

# 1차 merge: real_session_id 기반 사용자 연결
result = grouped_flow.merge(session_map[['real_session_id', 'user_id']],
                            on='real_session_id', how='left')

# 2차 merge: user_id 기반으로 디바이스/OS 정보 복구
result = result.merge(df[['user_id', 'device_id', 'osname']].drop_duplicates(),
                      on='user_id', how='left')

print("최종 result 테이블 구성 완료!")
print(f"행 수: {len(result):,}")

In [ ]:
print(f"고유 user_id 수: {result['user_id'].nunique():,}")
print(f"고유 device_id 수: {result['device_id'].nunique():,}")
print(f"고유 real_session_id 수: {result['real_session_id'].nunique():,}")

In [ ]:
print("\n user_id NULL 샘플")
display(result[result['user_id'].isnull()].head())

print("\n device_id NULL 샘플")
display(result[result['device_id'].isnull()].head())

In [ ]:
# 유저별 실제 세션 수 계산
user_session_count = (
    result.groupby('user_id')['real_session_id']
    .nunique()
    .reset_index(name='real_session_count')
)

# 실제 단발성 유저 추출
one_time_users = user_session_count[user_session_count['real_session_count'] == 1]


print(f"총 유저 수: {len(user_session_count):,}")
print(f"진짜 단발성 유저 수: {len(one_time_users):,}")
print(f"비율: {len(one_time_users) / len(user_session_count) * 100:.2f}%")

In [ ]:
# event_flow에서 첫 이벤트 추출
result['first_event'] = result['event_flow'].str.split(',').str[0]

# event_flow에서 마지막 이벤트 추출
result['last_event'] = result['event_flow'].str.split(',').str[-1]

result

In [ ]:
# 유저별 실제 앱 사용 횟수 계산
use_app_count = (
    result.groupby('user_id')['real_session_id']
    .nunique()
    .reset_index(name='use_app_count')
)

# 총 유저 수는 전처리 과정에서 덜어낸 총 유저 수: 203,080 이것과 같아야 함
print(f"총 유저 수: {len(use_app_count):,}")

In [ ]:
print("앱 사용 횟수 분포 통계:")
print(use_app_count['use_app_count'].describe().apply(lambda x: format(x, ',.2f')))

In [ ]:
print(f"1회 사용 유저 비율: {(use_app_count['use_app_count'] == 1).mean() * 100:.2f}%")
print(f"2회 사용 유저 비율: {(use_app_count['use_app_count'] == 2).mean() * 100:.2f}%")
print(f"3~4회 사용 유저 비율: {((use_app_count['use_app_count'] >= 3) & (use_app_count['use_app_count'] < 5)).mean() * 100:.2f}%")
print(f"5회 이상 사용 유저 비율: {(use_app_count['use_app_count'] >= 5).mean() * 100:.2f}%")
print(f"상위 1% 사용 횟수: {use_app_count['use_app_count'].quantile(0.99)}회")


In [ ]:
import re

flow_df = result.copy()  # 원본 보존

def compress_flow(flow):
    if pd.isna(flow):
        return flow
    # 콤마 기준으로 split
    events = flow.split(',')
    # 연속 중복 제거
    compressed = [events[0]]
    for e in events[1:]:
        if e != compressed[-1]:
            compressed.append(e)
    return ','.join(compressed)

# 새 컬럼 생성
flow_df['compressed_event_flow'] = flow_df['event_flow'].apply(compress_flow)

# 원하는 컬럼만 추출
flow_pattern_search_df = flow_df[[
    'device_id', 'user_id', 'real_session_id', 'compressed_event_flow'
]].rename(columns={'compressed_event_flow': 'event_flow'})

print("flow_pattern_search_df 생성 완료!")
print(f"총 행 수: {len(flow_pattern_search_df):,}")
flow_pattern_search_df

In [ ]:
print(len(result), len(flow_pattern_search_df))

In [ ]:
def still_has_duplicate(flow):
    if pd.isna(flow):
        return False
    events = flow.split(',')
    return any(events[i] == events[i+1] for i in range(len(events) - 1))

remaining_dup = flow_pattern_search_df['event_flow'].apply(still_has_duplicate).sum()

print(f"연속 중복 남아있는 세션 수 : {remaining_dup:,}")

In [ ]:
from prefixspan import PrefixSpan

In [ ]:
# 최소 지지도 설정
min_support = 1000

# 이벤트 흐름 데이터를 리스트 형태로 변환
sequences = flow_pattern_search_df['event_flow'].dropna().apply(lambda x: x.split(',')).tolist()

print("Data ready for PrefixSpan")
print(f"Total sessions: {len(sequences):,}")
print(f"Current min_support: {min_support:,}")

In [ ]:
ps = PrefixSpan(sequences)
ps.maxlen = 3

# 지지도 기반 패턴 추출 (지지도 내림차순 정렬)
patterns = ps.frequent(min_support)

print("PrefixSpan complete!")
print(f"Found patterns: {len(patterns):,}")

In [ ]:
# (1) 지지도 기준 내림차순 정렬
patterns_sorted = sorted(patterns, key=lambda x: x[0], reverse=True)

# (2) 길이 2 이상 + 모두 동일 이벤트(A→A) 패턴 제거
patterns_filtered = [
    (support, seq)
    for support, seq in patterns_sorted
    if len(seq) >= 2 and len(set(seq)) > 1
]

# # (3) Top 20로 자르기
# top_n = 20
# patterns_top = patterns_filtered[:top_n]   # 여기서 선언됨!
patterns_top = patterns_filtered


# (4) Top20 총 패턴 발생 수
total_pattern_count = sum([support for support, _ in patterns_top])

# (5) 이벤트 패턴 테이블 생성
rows = []
for rank, (support, seq) in enumerate(patterns_top, start=1):
    rows.append({
        'rank': rank,
        'pattern_count': support,
        'pattern_ratio(%)': round((support / total_pattern_count) * 100, 4),
        'pattern': ' → '.join(seq),
        'pattern_list': seq,
        'pattern_len': len(seq),
    })

event_pattern = pd.DataFrame(rows)

print("event_pattern 생성 완료!")

In [ ]:
step_3_pattern = event_pattern[(event_pattern["pattern_len"] == 3) & (event_pattern["pattern_ratio(%)"] >= 0.15)].reset_index(drop=True)
step_3_pattern

In [ ]:
step_2_pattern = event_pattern[(event_pattern["pattern_len"] == 2) & (event_pattern["pattern_ratio(%)"] >= 0.15)].reset_index(drop=True)
step_2_pattern

- 생키 다이어그램 관련 코드

In [ ]:
def count_users(required_events):
    """event_flow에 모든 이벤트가 포함된 유저 수 반환"""
    query = result_3['event_flow'].apply(
        lambda x: all(ev in x for ev in required_events)
    )
    return result_3[query]['user_id'].nunique()

# 전체 시작점
source = count_users(['launch_app'])

# 질문 화면 진입
b1_1 = count_users(['launch_app', 'view_questions_tap'])

# 질문 오픈
b1_2 = count_users(['launch_app', 'view_questions_tap', 'click_question_open'])

# 질문 시작
b1_3 = count_users([
    'launch_app', 'view_questions_tap',
    'click_question_open', 'click_question_start'
])

# 질문 시작 후 재탐색(회귀)
def count_revisit(df):
    count = 0
    for flow in df['event_flow']:
        seq = flow.split(',')
        # click_question_start → view_questions_tap 패턴 탐지
        for i in range(len(seq) - 1):
            if seq[i] == 'click_question_start' and seq[i+1] == 'view_questions_tap':
                count += 1
                break  # 한 유저당 한 번만 카운트
    return count

b1_3_1 = count_revisit(result_2)


# 질문 완료
b1_4 = count_users([
    'launch_app', 'view_questions_tap',
    'click_question_open', 'click_question_start',
    'complete_question'
])

# 질문 완료 후 채팅
b1_4_1 = count_users([
    'launch_app', 'view_questions_tap',
    'click_question_open', 'click_question_start',
    'complete_question',  "click_timeline_chat_start"
])

source, b1_1, b1_2, b1_3, b1_3_1, b1_4, b1_4_1


In [ ]:
# -------------------------------------------
# Branch 2 유저 수 계산 함수들
# -------------------------------------------

# 1) 보유 여부 기반 (result_3)
def count_has_events(events):
    return result_3[
        result_3['event_flow'].str.contains(events[0], na=False) &
        result_3['event_flow'].str.contains(events[1], na=False)
    ]['user_id'].nunique()


def count_has_all(events):
    cond = True
    for ev in events:
        cond &= result_3['event_flow'].str.contains(ev, na=False)
    return result_3[cond]['user_id'].nunique()


# 2) 순서 기반 (df_raw_2)
def count_ordered(events):
    cond = result_3['event_flow'].str.contains(events[0], na=False)
    prev_index = 0

    for e in events[1:]:
        cond &= result_3['event_flow'].str.contains(e, na=False)

    # 순서 비교
    order_users = []
    for _, row in result_3[cond].iterrows():
        seq = row['event_flow'].split(',')
        try:
            idx = [seq.index(ev) for ev in events]
            if idx == sorted(idx):
                order_users.append(row['user_id'])
        except:
            pass

    return len(set(order_users))


# -------------------------------------------
# Branch 2 숫자 계산
# -------------------------------------------

# b2_1 : launch_app & click_appbar_alarm_center 보유
b2_1 = count_has_events(['launch_app', 'click_appbar_alarm_center'])

# b2_1_1 : 순서 기반 (result_3)
b2_1_1 = count_ordered(['launch_app', 'click_appbar_alarm_center', 'view_questions_tap'])

# b2_2 : 보유 여부 기반 (result_3)
b2_2 = count_has_all(['launch_app', 'click_appbar_alarm_center', 'view_shop'])

# b2_3 : 보유 여부 + click_purchase
b2_3 = count_has_all(['launch_app', 'click_appbar_alarm_center', 'view_shop', 'click_purchase'])

# b2_5 : 보유 여부 + complete_purchase
b2_4 = count_has_all(['launch_app', 'click_appbar_alarm_center', 'view_shop', 'click_purchase', 'complete_purchase'])

print("▶ Branch 2 counts")
print("b2_1 :", b2_1)
print("b2_1_1 :", b2_1_1)
print("b2_2 :", b2_2)
print("b2_3 :", b2_3)
print("b2_4 :", b2_4)

In [ ]:
print("source :", source, "# 앱 실행")

# 브랜치 1
print("b1_1 :", b1_1, "# 질문 화면 진입")
print("b1_2 :", b1_2, "# 질문 오픈")
print("b1_3 :", b1_3, "# 질문 시작")
print("b1_3_1 :", b1_3_1, "# 질문화면회귀(되돌아감)")
print("b1_4 :", b1_4, "# 질문 완료")
print("b1_4_1 :", b1_4_1, "# 채팅 시작")

# 브랜치 2
print("\nb2_1 :", b2_1, "# 알림 센터 진입")
print("b2_1_1 :", b2_1_1, "# 알림 후 질문 화면 도달")
print("b2_2 :", b2_2, "# 상점 진입")
print("b2_3 :", b2_3, "# 구매 클릭")
print("b2_4 :", b2_4, "# 구매 완료")

In [ ]:
def conv(source, *steps):
    """
    Conversion 계산 함수
    source: 최초 시작 유저 수
    steps: 단계별 도착 유저 수 입력
    """
    values = [source] + list(steps)
    result = []

    for i in range(1, len(values)):
        prev = values[i-1]
        now = values[i]

        prev_ratio = now / prev * 100 if prev != 0 else 0
        total_ratio = now / source * 100

        result.append({
            "from → to": f"{i-1} → {i}",
            "count": now,
            "전환률(이전 대비)": f"{prev_ratio:.2f}%",
            "전환률(전체 대비)": f"{total_ratio:.2f}%"
        })

    return result

In [ ]:
source = 191107

result_branch1 = conv(source, b1_1, b1_2, b1_3, b1_3_1, b1_4, b1_4_1)
result_branch2 = conv(source, b2_1, b2_1_1, b2_2, b2_3, b2_4)

import pandas as pd

print("\nBranch 1 전환 분석")
display(pd.DataFrame(result_branch1))

print("\nBranch 2 전환 분석")
display(pd.DataFrame(result_branch2))

In [ ]:
import plotly.graph_objects as go

labels = [
    "앱 실행",
    "질문 화면", "질문 열기", "질문 시작", "되돌아감", "질문 완료", "채팅 진입",
    "알림", "알림 이후 질문 화면",
    "상점 진입", "구매 클릭", "구매 완료"
]
idx = {l:i for i,l in enumerate(labels)}

links = [
    ("앱 실행", "질문 화면", 104353),
    ("질문 화면", "질문 열기", 63857),
    ("질문 열기", "질문 시작", 33910),
    ("질문 시작", "질문 화면", 5482),
    ("질문 시작", "질문 완료", 24836),
    ("질문 완료", "채팅 진입", 5793),
    ("앱 실행", "알림", 136890),
    ("알림", "질문 화면", 52861),
    ("알림", "상점 진입", 13269),
    ("상점 진입", "구매 클릭", 5603),
    ("구매 클릭", "구매 완료", 1436)
]
source = [idx[s] for s,t,_ in links]
target = [idx[t] for s,t,_ in links]
value  = [v for _,_,v in links]

node_values = [191107,104353,63857,33910,5482,24836,5793,136890,52861,13269,5603,1436]
baseline = node_values[0]
hover_data = [
    [f"{node_values[i]:,}명",
     f"{node_values[i]/baseline*100:.2f}%",
     f"{100-(node_values[i]/baseline*100):.2f}%"]
    for i in range(len(labels))
]

# 수동 좌표 (labels와 길이 동일, 0~1 범위)
x = [0.10,0.25,0.45,0.6,0.75,0.9,0.2,0.35,0.45,0.55,0.80,0.35]
y = [0.50,0.35,0.30,0.30,0.40,0.5,0.65,0.70,0.70,0.70,0.70,0.70]

fig = go.Figure(data=[go.Sankey(
    arrangement='fixed',
    node=dict(
        label=labels,
        x=x, y=y,
        pad=30, thickness=30,
        color=[
            "#b2df8a","#1f78b4","#33a02c","#fb9a99","#e31a1c","#a6cee3","#1f78b4",
            "#cab2d6","#6a3d9a","#fdbf6f","#ff7f00","#ff0000"
        ],
        customdata=hover_data,
        hovertemplate="<b>%{label}</b><br>전체: %{customdata[0]}<br>전환률: %{customdata[1]}<br>이탈률: %{customdata[2]}<extra></extra>"
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        hovertemplate="전환수: %{value:,}명<extra></extra>",
        color="rgba(90,90,90,0.2)"
    )
)])

fig.update_layout(
    title="유저 행동 전환 Sankey Diagram (전환률 + 이탈률 포함)",
    font_size=13,
    width=1800, height=900
)

fig.write_html("/home/nuwba/Desktop/sankey_user_flow.html")

fig.show()
